In [0]:
from pyspark.sql.functions import to_date, first, col, round

In [0]:
df = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

In [0]:
df = df.withColumn("date", to_date("date"))

In [0]:
result_df = df.groupBy("date").pivot("moeda").agg(first("taxa")).orderBy("date", ascend=False)

In [0]:
result_df_values = result_df.select("*")

In [0]:
moedas = ['EUR', 'GBP', 'USD']

In [0]:
for moeda in moedas:
    result_df_values = result_df_values.withColumn(moeda, round(1/col(moeda), 4))

In [0]:
result_df = result_df.coalesce(1)
result_df_values = result_df_values.coalesce(1)

In [0]:
result_df.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/taxas_conversao")
    
result_df_values.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/valores_reais")